In [6]:
from rdkit import Chem
from rdkit.Chem import AllChem,Draw,Descriptors
from rdkit import rdBase
import math
import re

# check http://cheminformist.itmol.com/rdkit/file/tag


"""
Note
* enry with smiles string that has wild card element "*" will be removed, since it is not compatible with InChI handling in this script.
* entry with molecular formula that has "R" will be removed.
* entry that do not have both smiles and InChI strings will be removed
"""

'''
class class_candidate_mol:

    def __init__(self):
            self.pubchem_cid    =    0
            self.smiles    =    ()
            self.exact_mol_weight = 0
'''

# class to hold molecular info
class class_cmpd_from_sdf:
    def __init__(self):
            self.name = ""
            self.db_id    =    0
            self.smiles    =    ""
            self.exact_mol_weight = 0
            self.molecular_formula = ""
            self.inchi = ""
            self.inchi_key = ""
            self.list_CAS_registory_numbers = []
            self.list_HMDB_ID =[]
            self.link = ""
            self.rdkit_mol = Chem

#####
#  this is for Non lite file.
#
"""
list_id_to_avoid : list containing db id to avoid processing. Some entries has illegal inchi/smiles and return error
list_forbidden_element: include element in molecular formulae that is not compatible with RDkit or InChI standard. for instance R will cause error when converting to InChI
flag_forbid_wildcard_in_smiles : if 1 (default). entry whose smiles string cantain wild card element * will be ignored.

"""

# this function is to read sdf file from CheBI
def read_sdf_a2_ChEBI(sdf_filename ,list_id_to_avoid = [],list_forbidden_element = ["R"], flag_forbid_wildcard_in_smiles = 1 ):

    # read sdf file using RDkit function
    # suppl is a list of RDkit mols

    suppl = Chem.SDMolSupplier(sdf_filename)
    
    # initialize list that holds "class_cmpd_from_sdf" objects
    list_obj_candidate_mol = []

    count = 0
    count_valid = 0
    # iterate over RDkit mols
    for mol in suppl:
        if count%100 ==0 :print count

        # if sdf file content(mainly coordinates) is invalid or incompatible with RDkit, mol is None.
        if mol is not None :

            flag_valid_entry = 1

            # initialize class
            obj_cmpd_from_sdf = class_cmpd_from_sdf()

            # if "ChEBI Name" attribute is present in the mol,
            #   get the value and put in my object
            if "ChEBI Name" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.name = mol.GetProp("ChEBI Name")

            ###
            ### Note if thre is description about structures and properties, prioritize them.

            # ID.  Chebi provide id differently when downloading batch or separately ?????

            # if "ChEBI ID" attribute is present in the mol,
            #   get the value and put in my object
            if "ChEBI ID" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.db_id = str(mol.GetProp("ChEBI ID"))
            if "ID" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.db_id = str(mol.GetProp("ID"))

            # molecular formula related
            if "FORMULA" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.molecular_formula = str(mol.GetProp("FORMULA"))
            if "MF" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.molecular_formula = str(mol.GetProp("MF"))
            if "Formulae" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.molecular_formula = str(mol.GetProp("Formulae"))


            # inchi and related identifier
             
                """
             ^\d+?\.\d+?$ =
              # 文字列の先頭、任意の数字[0-9] : (一回以上の繰り返しと0回または１回= できるだけ多くのマッチ(greedy)), 任意の一文字, 
              任意の数字[0-9]:できるだけ多くのマッチ:文字列の末尾
              
                """
           
            
            if "InChI" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.inchi = str(mol.GetProp("InChI"))
            if "INCHIKEY" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.inchi_key = str(mol.GetProp("INCHIKEY"))
            if "InChIKey" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.inchi_key = str(mol.GetProp("InChIKey"))
            if "SMILES" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.smiles = str(mol.GetProp("SMILES"))
            if "Monoisotopic Mass" in list(mol.GetPropNames()):
                if re.match("^\d+?\.\d+?$", (mol.GetProp("Monoisotopic Mass"))) is not None:
                    monoisotopic_mass = float(mol.GetProp("Monoisotopic Mass"))
                    if monoisotopic_mass > 0.9 :
                        obj_cmpd_from_sdf.exact_mol_weight = monoisotopic_mass

            ####
            # !!!!!!!!!!!!!!!1
            # This part recognizes forbidden character in molecular formula.
            # You can use this function to exlude molecules containing particular element.
            #  now forbidden element is character, it hits H with He
            for ele in list_forbidden_element :
                if ele in obj_cmpd_from_sdf.molecular_formula:
                    flag_valid_entry = 0
                    print "entry", obj_cmpd_from_sdf.db_id , "was ignored, due to the presence of illegal character in molecular formula"


            # to avoid entry with smiles with wildcard element "*"
            forbidden_char_smiles = "*"
            if forbidden_char_smiles in obj_cmpd_from_sdf.smiles :
                flag_valid_entry = 0
            #
            # to avoid forbidden entry
            if obj_cmpd_from_sdf.db_id in list_id_to_avoid :
                flag_valid_entry = 0
                print "entry", obj_cmpd_from_sdf.db_id, "was ignored, due to prescribed forbidden entry"

            ######
            # checking at least SMILES or InChI is present
            if len(obj_cmpd_from_sdf.inchi) < 2  and   len(obj_cmpd_from_sdf.smiles) < 2:
                flag_valid_entry = 0


            if flag_valid_entry == 1:
                # get cas number
                if "CAS Registry Numbers" in list(mol.GetPropNames()):
                    # note some entries have multiple cas numbers.
                    str_cas = str(mol.GetProp("CAS Registry Numbers"))
                    obj_cmpd_from_sdf.list_CAS_registory_numbers = str_cas.splitlines()

                # get hmdb id
                if "HMDB Database Links" in list(mol.GetPropNames()):
                    # just like cas, entries might have multiple hmdb id
                    str_hmdb = str(mol.GetProp("HMDB Database Links"))
                    obj_cmpd_from_sdf.list_HMDB_ID = str_hmdb.splitlines()


                ####
                # now it is disabled since some entries have more than one exact mol weight
                # see CHEBI:3385
                """
                if "Monoisotopic Mass" in list(mol.GetPropNames()):
                    obj_cmpd_from_sdf.exact_mol_weight = float(mol.GetProp("Monoisotopic Mass"))
                """

                """
                # trying to get info from mol. but later rewritten. See below
                if len(obj_cmpd_from_sdf.smiles) < 1:
                    obj_cmpd_from_sdf.smiles = Chem.MolToSmiles(mol)

                if len(obj_cmpd_from_sdf.inchi) < 1  and mol is not None :
                    obj_cmpd_from_sdf.inchi = Chem.MolToInchi(mol)
                """

                """
                if len(obj_cmpd_from_sdf.inchi) < 1:
                    if mol is None : print "none"
                    obj_cmpd_from_sdf.inchi = Chem.MolToInchi(mol)
                """

                # try to create mol from inchi
                if len (obj_cmpd_from_sdf.inchi)  > 1 :
                    obj_cmpd_from_sdf.rdkit_mol = Chem.MolFromInchi(obj_cmpd_from_sdf.inchi)


                ## This part cause error with CHEBI:36973 - graphene

                # if inchi is not available, use smiles
                if len (obj_cmpd_from_sdf.inchi) < 2 :
                    obj_cmpd_from_sdf.rdkit_mol = Chem.MolFromSmiles(obj_cmpd_from_sdf.smiles)

                    if obj_cmpd_from_sdf.rdkit_mol != None:
                        print obj_cmpd_from_sdf.smiles
                        # add inchi
                        obj_cmpd_from_sdf.inchi = Chem.MolToInchi(obj_cmpd_from_sdf.rdkit_mol)
                        #add inchi key
                        obj_cmpd_from_sdf.inchi_key = Chem.InchiToInchiKey(obj_cmpd_from_sdf.inchi)


                if obj_cmpd_from_sdf.rdkit_mol == None :
                    obj_cmpd_from_sdf.rdkit_mol = Chem.MolFromSmiles(obj_cmpd_from_sdf.smiles)



                if obj_cmpd_from_sdf.exact_mol_weight < 1 :
                    if obj_cmpd_from_sdf.rdkit_mol is not None :
                        obj_cmpd_from_sdf.exact_mol_weight = Descriptors.ExactMolWt(obj_cmpd_from_sdf.rdkit_mol)



                # Note !!!!!!!!!!!!
                # Chebi sdf itself lacks stereo info in SDF, its better to retain Inchi Directly from Inchi string

                # Finally adding my object to list.
                # in this version, only entries that give valid rdkit mol will be kept
                if obj_cmpd_from_sdf.rdkit_mol is not None:
                    list_obj_candidate_mol.append(obj_cmpd_from_sdf)
                    count_valid = count_valid  + 1

        count = count + 1

    print "TOTAL" , count
    print "TOTAL VALID" , count_valid
    # return the list of "class_cmpd_from_sdf" object".
    return list_obj_candidate_mol

    
def read_sdf_a2_chemspider(sdf_filename):
    suppl = Chem.SDMolSupplier(sdf_filename)

    list_obj_candidate_mol = []

    for mol in suppl:

        print "name",mol.GetProp("_Name")
        print "< name"
        obj_cmpd_from_sdf = class_cmpd_from_sdf()
        #Draw.MolToFile(mol,'cdk2_mol1.png')
        my_mass  = Descriptors.ExactMolWt(mol)
        obj_cmpd_from_sdf.name = mol.GetProp("_Name")
        obj_cmpd_from_sdf.smiles =  Chem.MolToSmiles(mol)
        obj_cmpd_from_sdf.exact_mol_weight = float(Descriptors.ExactMolWt(mol))
        obj_cmpd_from_sdf.db_id = int(mol.GetProp("CSID"))
        obj_cmpd_from_sdf.inchi = str(mol.GetProp("InChI"))
        obj_cmpd_from_sdf.link = str(mol.GetProp("CSURL"))

        list_obj_candidate_mol.append(    obj_cmpd_from_sdf )

    print "-----------------"

    # example with aspirin
    '''
    aspirin = suppl[0]
    for key in aspirin.GetPropNames():
        value = aspirin.GetProp(key)
        print ">",key
        print value
    '''

    # accessing pubchemcid ---------------------------------------


    return list_obj_candidate_mol



def get_candidate_in_db_by_mass(query_mass, ppm_tol, list_mol):

    list_candidate_matched =[]
    for obj in list_mol:
        if obj.exact_mol_weight > 0 and   (math.fabs(((query_mass - obj.exact_mol_weight)/obj.exact_mol_weight)*1000000)<ppm_tol):
            list_candidate_matched.append(obj)
    return list_candidate_matched



def write_tsv(query_mass, ppm_tol, list_mol):



    header = "ID    NAME    InChIKey    InChI   "
    return list_candidate_matched


def print_hhh(input):
    print "hhh"
    return input + 1

read_sdf_a2_ChEBI('/Users/minato/JupiterNotebookFolder/Python Chemo/ChEBI_complete_3star_subset_for_test.sdf', list_id_to_avoid = [],list_forbidden_element = ["R"], flag_forbid_wildcard_in_smiles = 1 )



0
entry CHEBI:598 was ignored, due to the presence of illegal character in molecular formula
entry CHEBI:1624 was ignored, due to the presence of illegal character in molecular formula
entry CHEBI:1722 was ignored, due to the presence of illegal character in molecular formula
entry CHEBI:2220 was ignored, due to the presence of illegal character in molecular formula
TOTAL 53
TOTAL VALID 47


[<__main__.class_cmpd_from_sdf instance at 0x10c389368>,
 <__main__.class_cmpd_from_sdf instance at 0x11dafd908>,
 <__main__.class_cmpd_from_sdf instance at 0x11db020e0>,
 <__main__.class_cmpd_from_sdf instance at 0x11db02488>,
 <__main__.class_cmpd_from_sdf instance at 0x11db02878>,
 <__main__.class_cmpd_from_sdf instance at 0x11db02fc8>,
 <__main__.class_cmpd_from_sdf instance at 0x11db00128>,
 <__main__.class_cmpd_from_sdf instance at 0x11db000e0>,
 <__main__.class_cmpd_from_sdf instance at 0x11db00200>,
 <__main__.class_cmpd_from_sdf instance at 0x11db003f8>,
 <__main__.class_cmpd_from_sdf instance at 0x11db00320>,
 <__main__.class_cmpd_from_sdf instance at 0x11db00518>,
 <__main__.class_cmpd_from_sdf instance at 0x11db005f0>,
 <__main__.class_cmpd_from_sdf instance at 0x11db006c8>,
 <__main__.class_cmpd_from_sdf instance at 0x11db007e8>,
 <__main__.class_cmpd_from_sdf instance at 0x11db008c0>,
 <__main__.class_cmpd_from_sdf instance at 0x11db00998>,
 <__main__.class_cmpd_from_sdf 

In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem,Draw,Descriptors
from rdkit import rdBase
import math
import re
import pubchempy as pcp

# class to hold molecular info
class class_cmpd_from_sdf:
    def __init__(self):
            self.name = ""
            self.db_id    =    0
            self.smiles    =    ""
            self.exact_mol_weight = 0
            self.molecular_formula = ""
            self.inchi = ""
            self.inchi_key = ""
            self.list_CAS_registory_numbers = []
            self.list_HMDB_ID =[]
            self.link = ""
            self.rdkit_mol = Chem

        
def read_sdf_a2_ChEBI(sdf_filename ,list_id_to_avoid = [],list_forbidden_element = ["R"], flag_forbid_wildcard_in_smiles = 1 ):

    suppl = Chem.SDMolSupplier('/Users/minato/JupiterNotebookFolder/Python Chemo/ChEBI_complete_3star_subset_for_test.sdf')
    
 # initialize list that holds "class_cmpd_from_sdf" objects
    list_obj_candidate_mol = []
    count = 0
    count_valid = 0
    
    for mol in suppl:
        if count%100 ==0 :print count

        # if sdf file content(mainly coordinates) is invalid or incompatible with RDkit, mol is None.
        if mol is not None :

            flag_valid_entry = 1

            # initialize class
            obj_cmpd_from_sdf = class_cmpd_from_sdf()

            # if "ChEBI Name" attribute is present in the mol, get the value and put in my object
            if "ChEBI Name" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.name = mol.GetProp("ChEBI Name")

            #Note if thre is description about structures and properties, prioritize them.
            # if "ChEBI ID" attribute is present in the mol, get the value and put in my object
            if "ChEBI ID" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.db_id = str(mol.GetProp("ChEBI ID"))
            if "ID" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.db_id = str(mol.GetProp("ID"))

            # molecular formula related
            if "FORMULA" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.molecular_formula = str(mol.GetProp("FORMULA"))
            if "MF" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.molecular_formula = str(mol.GetProp("MF"))
            if "Formulae" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.molecular_formula = str(mol.GetProp("Formulae"))
           
        # inchi and related identifier
            if "InChI" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.inchi = str(mol.GetProp("InChI"))
            if "INCHIKEY" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.inchi_key = str(mol.GetProp("INCHIKEY"))
            if "InChIKey" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.inchi_key = str(mol.GetProp("InChIKey"))
            if "SMILES" in list(mol.GetPropNames()):
                obj_cmpd_from_sdf.smiles = str(mol.GetProp("SMILES"))
            if "Monoisotopic Mass" in list(mol.GetPropNames()):
                if re.match("^\d+?\.\d+?$", (mol.GetProp("Monoisotopic Mass"))) is not None:
                    monoisotopic_mass = float(mol.GetProp("Monoisotopic Mass"))
                    if monoisotopic_mass > 0.9 :
                        obj_cmpd_from_sdf.exact_mol_weight = monoisotopic_mass
        
        # This part recognizes forbidden character in molecular formula.
            for ele in list_forbidden_element :
                if ele in obj_cmpd_from_sdf.molecular_formula:
                    flag_valid_entry = 0
                    print "entry", obj_cmpd_from_sdf.db_id , "was ignored, due to the presence of illegal character in molecular formula"

         # to avoid entry with smiles with wildcard element "*"
            forbidden_char_smiles = "*"
            if forbidden_char_smiles in obj_cmpd_from_sdf.smiles :
                flag_valid_entry = 0
            
            # to avoid forbidden entry
            if obj_cmpd_from_sdf.db_id in list_id_to_avoid :
                flag_valid_entry = 0
                print "entry", obj_cmpd_from_sdf.db_id, "was ignored, due to prescribed forbidden entry"

            # checking at least SMILES or InChI is present
            if len(obj_cmpd_from_sdf.inchi) < 2  and   len(obj_cmpd_from_sdf.smiles) < 2:
                flag_valid_entry = 0

            if flag_valid_entry == 1:
                # get cas number
                if "CAS Registry Numbers" in list(mol.GetPropNames()):
                    # note some entries have multiple cas numbers.
                    str_cas = str(mol.GetProp("CAS Registry Numbers"))
                    obj_cmpd_from_sdf.list_CAS_registory_numbers = str_cas.splitlines()

                # get hmdb id
                if "HMDB Database Links" in list(mol.GetPropNames()):
                    # just like cas, entries might have multiple hmdb id
                    str_hmdb = str(mol.GetProp("HMDB Database Links"))
                    obj_cmpd_from_sdf.list_HMDB_ID = str_hmdb.splitlines()
                    
                # try to create mol from inchi
                if len (obj_cmpd_from_sdf.inchi)  > 1 :
                    obj_cmpd_from_sdf.rdkit_mol = Chem.MolFromInchi(obj_cmpd_from_sdf.inchi)

                # if inchi is not available, use smiles
                if len (obj_cmpd_from_sdf.inchi) < 2 :
                    obj_cmpd_from_sdf.rdkit_mol = Chem.MolFromSmiles(obj_cmpd_from_sdf.smiles)

                    if obj_cmpd_from_sdf.rdkit_mol != None:
                        print obj_cmpd_from_sdf.smiles
                        # add inchi
                        obj_cmpd_from_sdf.inchi = Chem.MolToInchi(obj_cmpd_from_sdf.rdkit_mol)
                        #add inchi key
                        obj_cmpd_from_sdf.inchi_key = Chem.InchiToInchiKey(obj_cmpd_from_sdf.inchi)

                if obj_cmpd_from_sdf.rdkit_mol == None :
                    obj_cmpd_from_sdf.rdkit_mol = Chem.MolFromSmiles(obj_cmpd_from_sdf.smiles)

                if obj_cmpd_from_sdf.exact_mol_weight < 1 :
                    if obj_cmpd_from_sdf.rdkit_mol is not None :
                        obj_cmpd_from_sdf.exact_mol_weight = Descriptors.ExactMolWt(obj_cmpd_from_sdf.rdkit_mol)


                # Adding objects to list, only entries with valid rdkit mol will be kept
                if obj_cmpd_from_sdf.rdkit_mol is not None:
                    list_obj_candidate_mol.append(obj_cmpd_from_sdf)
                   
                    count_valid = count_valid  + 1

        count = count + 1

    print "TOTAL" , count
    print "TOTAL VALID" , count_valid
    # return the list of "class_cmpd_from_sdf" object".
    return list_obj_candidate_mol



read_sdf_a2_ChEBI(df ,list_id_to_avoid = [],list_forbidden_element = ["R"], flag_forbid_wildcard_in_smiles = 1 )
list_mol = list_obj_candidate_mol

""""
Chemspider ver. 
def read_sdf_a2_chemspider(sdf_filename):
    suppl = Chem.SDMolSupplier(sdf_filename)

    list_obj_candidate_mol = []

    for mol in suppl:

        print "name",mol.GetProp("_Name")
        print "< name"
        obj_cmpd_from_sdf = class_cmpd_from_sdf()
        #Draw.MolToFile(mol,'cdk2_mol1.png')
        my_mass  = Descriptors.ExactMolWt(mol)
        obj_cmpd_from_sdf.name = mol.GetProp("_Name")
        obj_cmpd_from_sdf.smiles =  Chem.MolToSmiles(mol)
        obj_cmpd_from_sdf.exact_mol_weight = float(Descriptors.ExactMolWt(mol))
        obj_cmpd_from_sdf.db_id = int(mol.GetProp("CSID"))
        obj_cmpd_from_sdf.inchi = str(mol.GetProp("InChI"))
        obj_cmpd_from_sdf.link = str(mol.GetProp("CSURL"))

        list_obj_candidate_mol.append(    obj_cmpd_from_sdf )

    print "-----------------"

    # example with aspirin
    '''
    aspirin = suppl[0]
    for key in aspirin.GetPropNames():
        value = aspirin.GetProp(key)
        print ">",key
        print value
    '''

    return list_obj_candidate_mol

""""

df = '/Users/minato/JupiterNotebookFolder/Python Chemo/ChEBI_16865.sdf'
read_sdf_a2_ChEBI(df ,list_id_to_avoid = [],list_forbidden_element = ["R"], flag_forbid_wildcard_in_smiles = 1 )
        list_obj_candidate_mol.append(obj_cmpd_from_sdf )

def get_candidate_in_db_by_mass(query_mass, ppm_tol, list_mol):
    list_candidate_matched = []
    for obj in list_mol:
        if obj.exact_mol_weight > 0 and   (math.fabs(((query_mass - obj.exact_mol_weight)/obj.exact_mol_weight)*1000000)<ppm_tol):
            list_candidate_matched.append(obj)
    return list_candidate_matched

def write_tsv(query_mass, ppm_tol, list_mol):
    header = "ID    NAME    InChIKey    InChI   "
    return list_candidate_matched


def print_hhh(input):
    print "hhh"
    return input + 1

aspirin = suppl[0]
    for key in aspirin.GetPropNames():
        value = aspirin.GetProp(key)
        print ">",key
        print value

SyntaxError: EOL while scanning string literal (<ipython-input-8-11b3cf1e89d7>, line 188)